# DDR Assignment 5.2

### Team Members - Group 17

- Nidhi Patni
- Aravind Venugopal
- Jacky Xie
- Ayush Tripathi

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DDR-Assignment-5.2" data-toc-modified-id="DDR-Assignment-5.2-1">DDR Assignment 5.2</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Team-Members---Group-17" data-toc-modified-id="Team-Members---Group-17-1.0.1">Team Members - Group 17</a></span></li></ul></li><li><span><a href="#1.-Create-a-MongoDB-database-called-&quot;amazon&quot;." data-toc-modified-id="1.-Create-a-MongoDB-database-called-&quot;amazon&quot;.-1.1">1. Create a MongoDB database called "amazon".</a></span></li><li><span><a href="#2.-Read-&quot;reviews_electronics.16.json&quot;-and-uploads-each-review-as-a-separate-document-to-the-collection-&quot;reviews&quot;-in-the-database-&quot;amazon&quot;." data-toc-modified-id="2.-Read-&quot;reviews_electronics.16.json&quot;-and-uploads-each-review-as-a-separate-document-to-the-collection-&quot;reviews&quot;-in-the-database-&quot;amazon&quot;.-1.2">2. Read "reviews_electronics.16.json" and uploads each review as a separate document to the collection "reviews" in the database "amazon".</a></span></li><li><span><a href="#3.-Uses-MongoDB's-map-reduce-function-to-build-a-new-collection-&quot;avg_scores&quot;-that-averages-review-scores-by-product-(&quot;asin&quot;).-Print-the-first-100-entries-of-&quot;avg_scores&quot;-to-screen." data-toc-modified-id="3.-Uses-MongoDB's-map-reduce-function-to-build-a-new-collection-&quot;avg_scores&quot;-that-averages-review-scores-by-product-(&quot;asin&quot;).-Print-the-first-100-entries-of-&quot;avg_scores&quot;-to-screen.-1.3">3. Uses MongoDB's map reduce function to build a new collection "avg_scores" that averages review scores by product ("asin"). Print the first 100 entries of "avg_scores" to screen.</a></span></li><li><span><a href="#4.-Use-MongoDB's-map-reduce-function-to-build-a-new-collection-&quot;weighted_avg_scores&quot;-that-averages-review-scores-by-product-(&quot;asin&quot;),-weighted-by-the-number-of-helpful-votes-(The-base-weight-is-1-and-for-every-additional-helpful-vote-add-1-to-weight." data-toc-modified-id="4.-Use-MongoDB's-map-reduce-function-to-build-a-new-collection-&quot;weighted_avg_scores&quot;-that-averages-review-scores-by-product-(&quot;asin&quot;),-weighted-by-the-number-of-helpful-votes-(The-base-weight-is-1-and-for-every-additional-helpful-vote-add-1-to-weight.-1.4">4. Use MongoDB's map reduce function to build a new collection "weighted_avg_scores" that averages review scores by product ("asin"), weighted by the number of helpful votes (The base weight is 1 and for every additional helpful vote add 1 to weight.</a></span></li></ul></li></ul></div>

In [1]:
import pymongo, json
from tqdm import tqdm
from datetime import datetime, date
import pandas as pd
from bson.code import Code

## 1. Create a MongoDB database called "amazon".

In [14]:
# Initialize MongoDB client
client = pymongo.MongoClient("mongodb://localhost:27017/")

# create the database
db = client["amazon"]

# create the collection
col = db["reviews"]

## 2. Read "reviews_electronics.16.json" and uploads each review as a separate document to the collection "reviews" in the database "amazon".

In [2]:
# Parse the json file into a python object to iterate over
data = pd.read_json('reviews_electronics.16.json', typ = 'series', lines = True)
list_data = data.tolist()

In [50]:
# Convert date strings to datetime objects supported by MongoDB
for review in tqdm(list_data):
    review['reviewTime'] = datetime.strptime(review['reviewTime'],'%m %d, %Y')
    review['unixReviewTime'] = datetime.utcfromtimestamp(review['unixReviewTime'])

100%|███████████████████████████████████████████████████████████████████████| 324546/324546 [00:06<00:00, 46787.36it/s]


In [51]:
# Bulk insert the data as documents into the local MongoDB instance
result = col.insert_many(list_data)

## 3. Uses MongoDB's map reduce function to build a new collection "avg_scores" that averages review scores by product ("asin"). Print the first 100 entries of "avg_scores" to screen.

In [65]:
# Define Map, reduce and query functions
map_ = Code("""function(){emit(this.asin, this.overall)}""")
reduce = Code("""function(key,value){
    return Array.avg(value)
}""")
query = {"overall":{'$exists':1}}
results = col.map_reduce(map_, reduce,"avg_scores" ,query = query)

In [68]:
# Print out the top 100 results
for res in results.find().limit(100):
    print(res)

{'_id': '0132793040', 'value': 5.0}
{'_id': 'B00E4KP4W6', 'value': 4.545454545454546}
{'_id': 'B00E4KP8VI', 'value': 5.0}
{'_id': 'B00E4KPMC8', 'value': 2.0}
{'_id': 'B00E4KQ5C4', 'value': 5.0}
{'_id': 'B00E4KQ9GG', 'value': 3.2857142857142856}
{'_id': 'B00E4KQ9K2', 'value': 5.0}
{'_id': 'B00E4KQD4E', 'value': 4.0}
{'_id': 'B00E4KZBX8', 'value': 4.0}
{'_id': 'B00E4KZDJ0', 'value': 5.0}
{'_id': 'B00E4L35DA', 'value': 4.0}
{'_id': 'B00E4L3N9Q', 'value': 4.0}
{'_id': 'B00E4L48EA', 'value': 5.0}
{'_id': 'B00E4L7FLI', 'value': 1.0}
{'_id': 'B00E4L7TS2', 'value': 4.0}
{'_id': 'B00E4LAL82', 'value': 3.0}
{'_id': 'B00E4LBZZK', 'value': 5.0}
{'_id': 'B00E4LF2Z4', 'value': 4.333333333333333}
{'_id': 'B00E4LFP0G', 'value': 4.444444444444445}
{'_id': 'B00E4LFWWW', 'value': 4.4}
{'_id': 'B00E4LGTVU', 'value': 4.195658625514055}
{'_id': 'B00E4LGTXS', 'value': 3.923076923076923}
{'_id': 'B00E4LGVYA', 'value': 3.272727272727273}
{'_id': 'B00E4LGWLW', 'value': 3.5}
{'_id': 'B00E4LGXL6', 'value': 5.0}
{

## 4. Use MongoDB's map reduce function to build a new collection "weighted_avg_scores" that averages review scores by product ("asin"), weighted by the number of helpful votes (The base weight is 1 and for every additional helpful vote add 1 to weight.
e.g. a "[3, 5]" value on "helpful" column should use 3 + 1 = 4 as weight, 3 being the additional votes and 1 being the base weight). Print the first 100 entries of "weighted_avg_scores" to screen.

In [29]:
# Define Map, reduce and query functions
map_ = Code("""function(){emit(this.asin, {weighted:this.overall*(this.helpful[0]+1),weight:(this.helpful[0]+1)})}""")
reduce = Code("""function (key, values) {
    var a = {"weighted":0,"weight":0};
    for (var i=1; i < values.length; i++){
        var b = values[i];
        
        a.weighted += b.weighted;
        a.weight+= b.weight;
    }

    return a;
}""")
finalize = Code("""function (key, value){ 
    return (value.weighted/value.weight);
}""")
query = {"overall":{'$exists':1}}
results = col.map_reduce(map_, reduce,"weighted_avg_scores" ,query = query,finalize = finalize)


# print out top 100 results
for res in results.find().limit(100):
    print(res)

{'_id': '0132793040', 'value': 5.0}
{'_id': 'B00E4KP4W6', 'value': 4.647058823529412}
{'_id': 'B00E4KP8VI', 'value': 5.0}
{'_id': 'B00E4KPMC8', 'value': 2.0}
{'_id': 'B00E4KQ5C4', 'value': 5.0}
{'_id': 'B00E4KQ9GG', 'value': 3.642857142857143}
{'_id': 'B00E4KQ9K2', 'value': 5.0}
{'_id': 'B00E4KQD4E', 'value': 4.0}
{'_id': 'B00E4KZBX8', 'value': 4.0}
{'_id': 'B00E4KZDJ0', 'value': 5.0}
{'_id': 'B00E4L35DA', 'value': 2.8181818181818183}
{'_id': 'B00E4L3N9Q', 'value': 4.0}
{'_id': 'B00E4L48EA', 'value': 5.0}
{'_id': 'B00E4L7FLI', 'value': 1.0}
{'_id': 'B00E4L7TS2', 'value': 4.0}
{'_id': 'B00E4LAL82', 'value': 3.0}
{'_id': 'B00E4LBZZK', 'value': 5.0}
{'_id': 'B00E4LF2Z4', 'value': 4.083333333333333}
{'_id': 'B00E4LFP0G', 'value': 4.454545454545454}
{'_id': 'B00E4LFWWW', 'value': 5.0}
{'_id': 'B00E4LGTVU', 'value': 4.379679144385027}
{'_id': 'B00E4LGTXS', 'value': 3.857142857142857}
{'_id': 'B00E4LGVYA', 'value': 3.9411764705882355}
{'_id': 'B00E4LGWLW', 'value': 3.9}
{'_id': 'B00E4LGXL6', 

<img src="https://userscontent2.emaze.com/images/34cd88bd-153a-4481-8fef-2116b9fbd8ad/ae4c5bb012f3b8b25ee145d04be7ecc5.png" alt="drawing" width="120" align="centre"/>